In [1]:
import numpy as np

# 1. Simulate data

$$y_t = u'_t \phi + x'_t \beta + \epsilon_t, \epsilon_t \stackrel{\text{i.i.d}}{\sim} \mathcal{N}(0, \sigma^{2}) $$
In our case, as the dimension of U is l = 0, we have 
$$y_t = x'_t \beta + \epsilon_t, \epsilon_t \stackrel{\text{i.i.d}}{\sim} \mathcal{N}(0, \sigma^{2}) $$

In [2]:
l = 0
k = 100
T = 200

In [3]:
a, b, A, B = 1, 1, 1, 1

### Draw initial $\sigma²$

In [4]:
def draw_inital_sigma_squared(X, beta, Ry, T=T):
    return ((1/Ry) - 1) * (1/T) * sum([(beta.T @ x)**2 for x in X])

### Draw initial $\beta$

In [5]:
def draw_initial_beta(s, k=k):
    beta = np.random.normal(size=k)
    z = np.random.choice(range(k), size=k-s, replace=False)
    beta[z] = 0
    return beta, z

### Simulate X

In [6]:
from scipy.linalg import toeplitz

In [7]:
rho = 0.75

In [8]:
def draw_X(rho=rho, k=k, T=T):
    correlation_matrix = toeplitz(rho ** np.arange(0, k))
    return np.random.multivariate_normal(np.zeros(k), correlation_matrix, size=T)

### Simulate Y

In [9]:
def draw_Y(X, beta, sigma_squared):
    return X @ beta + np.random.normal(0, sigma_squared, size=(X @ beta).shape[0])

In [10]:
def draw_dataset(Ry, s, rho=rho, k=k, T=T):
    X = draw_X()
    beta, z = draw_initial_beta(s, k=k)
    sigma_squared = draw_inital_sigma_squared(X, beta, Ry, T=T)
    Y = draw_Y(X, beta, sigma_squared)
    return Y, X, beta, sigma_squared, z

# 2. Draw from posteriors

In [11]:
def inverse_matrix(A):
    try:
        return np.linalg.solve(A, np.eye(A.shape[0]))
    except:
        return A

In [15]:
def draw_tildes(Y, X, beta, gamma, z):
    X_tilde = X[:, z != 0]
    beta_tilde = beta[z != 0]
    W_tilde = X_tilde.T @ X_tilde + (1/gamma**2) * np.eye(np.sum(z))
    Y_tilde = Y 
    beta_tilde_hat = inverse_matrix(W_tilde) @ X_tilde.T @ Y_tilde
    return X_tilde, beta_tilde, W_tilde, Y_tilde, beta_tilde_hat

### (I). Draw from $R²$

In [ ]:
def draw_r2(X, z, beta, sigma_squared, k=k, a=a, b=b, A=A, B=B):
    s_z = np.sum(z)
    v_x_bar = (1/k) * sum([np.var(x) for x in X])
    grid = np.concatenate((np.arange(0, 0.1, 0.001), np.arange(0.1, 0.9, 0.01), np.arange(0.9, 1.001, 0.001)))
    unnormalized_posterior = np.zeros((len(grid), len(grid)))
    
    for i in range(len(grid)):
        for j in range(len(grid)):
            q = grid[i]
            r2 = grid[j]
            
            exp_term = np.exp((-1/(2*sigma_squared)) * ((k * v_x_bar * q * (1 - r2)) / r2) * (beta.T @ np.diag(z) @ beta))
            q_term = q ** (s_z + s_z/2 + a - 1) * (1-q) ** (k - s_z + b - 1)
            r2_term = r2 ** (A - 1 - s_z/2) ** (1-r2) ** (s_z/2 + B - 1)
            unnormalized_posterior[i, j] = exp_term * q_term * r2_term
    
    # normalize the posterior
    normalized_posterior /= np.sum(unnormalized_posterior)
    
    # Draw samples from the normalized posterior
    i, j = np.random.multinomial(1, normalized_posterior.flatten(), size=2)
    i, j = np.argmax(i), np.argmax(j)
    q = grid[i]
    r2 = grid[j]
    return r2, q

### (III). Draw from z

In [ ]:
def draw

### (IV). Draw from $\sigma²$

In [11]:
import scipy.stats as stats

In [12]:
def draw_sigma(Y, X, beta, gamma, z):
    X_tilde, beta_tilde, W_tilde, Y_tilde, beta_tilde_hat = draw_tildes(Y, X, beta, gamma, z)
    variance = (1/2) * (Y_tilde.T @ Y_tilde - beta_tilde_hat.T @ W_tilde @ beta_tilde_hat)
    return stats.invgamma.rvs(T/2, variance, size=1)

### (V). Draw from $\tilde{\beta}$

In [13]:
def draw_beta_tilde(Y, U, X, phi, beta, gamma, z, sigma_squared):
    X_tilde, beta_tilde, W_tilde, Y_tilde, beta_tilde_hat = draw_tildes(Y, U, X, phi, beta, gamma, z)
    mean = inverse_matrix(W_tilde) @ X_tilde.T @ Y_tilde
    variance = sigma_squared * inverse_matrix(W_tilde)
    return np.random.multivariate_normal(mean, variance, size=X_tilde.shape[1])

In [94]:
import numpy as np

# Create a 2D array of probabilities
pvals = np.array([[0.2, 0.3, 0.5], [0.4, 0.3, 0.3]])
pvals /= np.sum(pvals)

# Draw 10 samples from the multinomial distribution
a, b = np.random.multinomial(1, pvals.flatten(), size=2)
a = np.argmax(a)
b = np.argmax(b)
a, b

(3, 2)